# Enviromental Setup and Library import

In [0]:
import os
import re
import sys
import csv
import time
import nltk
import math
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from scipy.stats import uniform
from scipy.stats import randint
from sklearn import metrics
from sklearn import tree
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
drive.mount('/content/gdrive')
nltk.download('wordnet')
nltk.download('stopwords')
!unzip -qq gdrive/My\ Drive/dataset.zip
stop = stopwords.words('english')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
unzip:  cannot find or open gdrive/My Drive/dataset.zip, gdrive/My Drive/dataset.zip.zip or gdrive/My Drive/dataset.zip.ZIP.


# Read data from .txt files and preprocessing

In [0]:
def read_data(tp):
    print('Reading %s data to Pandas Dataframe...' %(tp))
    path = 'dataset/' + tp + '/'
    labels = {'pos':1, 'neg':0}
  
    df = pd.DataFrame()
    start = time.time()
    if tp == 'train':
        for p in labels.keys():
            list_of_files = os.listdir(path + p + '/')
            for file in list_of_files:
                f = open(path + p + '/' + file, 'r')
                txt = f.read()
                df = df.append([[txt, labels[p]]], ignore_index = True)
                f.close()
    
    else:
        list_of_files = os.listdir(path)
        for file in list_of_files:
            f = open(path + file, 'r')
            txt = f.read()
            id = int(re.sub('.txt', '', file))
            df = df.append([[txt, id+100000]], ignore_index = True)
            f.close()
      
    df.columns = ['text', 'sentiment']
    print('Done.')
    end = time.time()
    print('Time elapsed on reading is', end - start,'\n')
    return df
np.random.seed(88)   
tr_raw = read_data('train')
te_raw = read_data('test')
df_raw = pd.concat([tr_raw, te_raw], axis = 0)
df_raw.describe()



Reading train data to Pandas Dataframe...


FileNotFoundError: ignored

In [0]:
#nltk.download('wordnet')
def preprocess(text): 
    text = text.lower().split()
#     stops = set(stopwords.words("english"))
#     text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"^https?:\/\/.*[\r\n]*","", text)
    text = text.split()
    lemma = nltk.wordnet.WordNetLemmatizer()
    stemmed_words = [lemma.lemmatize(word) for word in text]
    text = " ".join(stemmed_words)
    return text
  
df = df_raw.copy()
df['text'] = df['text'].map(lambda x: preprocess(x))

# Dataset Split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[df['sentiment']<2]['text'], df.loc[df['sentiment']<2]['sentiment'], train_size=0.75, test_size=0.25)
X_train_total = df.loc[df['sentiment']<2]['text'].copy()
y_train_total = df.loc[df['sentiment']<2]['sentiment'].copy()
X_test.head()

19692    dave is going through a divorce and his mind w...
11457    a wonderful free flowing often lyrical film th...
13050    never even knew this movie existed until i fou...
16393    watching that lady in ermine i wa wondering wh...
21348    2 star for kay francis - - she wonderful ! and...
Name: text, dtype: object

# Feature Extraction and Model Pipeline

In [0]:
def csv_exp(result, name):
  csv = result.to_csv(index=False)
  f = open(name+'.csv','w')
  f.write(csv)
  f.close()
  

In [0]:
def accuracy(arr1, arr2):
  return 1 - sum(abs(arr1 - arr2))/len(arr1)
seed = 88

In [0]:
def cv_tabular(result):
  dic = {}
  for x in result['params'][0]:
    dic[x] = []
  for x in result['params']:
    for key in dic.keys():
      dic[key].append(x[key])
  cv_df = pd.DataFrame(data = dic)
  cv_df['mean_test_score'] = result['mean_test_score']
  cv_df['std_test_score'] = result['std_test_score']
  cv_df['mean_fit_time'] = result['mean_fit_time']
  cv_df = cv_df.sort_values(by = 'mean_test_score')
  return cv_df

## Logistic Regression Estimator

In [0]:
logreg = Pipeline([
    ('vect', CountVectorizer(binary = True)),
    ('tfidf', TfidfTransformer()),
    ('norm', Normalizer()),
    ('clf', LogisticRegression(C=10)),
])

 

In [0]:
seed = 130
params_log = {"vect__ngram_range": [(1,1),(2,2),(3,3)],
              "vect__binary":[True, False],
              "vect__max_features":[None, 50000, 100000],
              "clf__penalty": ["l2", "l1"]}
logreg_cv = GridSearchCV(logreg, param_grid = params_log, cv=2, verbose = 10, n_jobs = -1)
logreg_cv.fit(X_train, y_train)
y_pred = logreg_cv.predict(X_test)
print('My best accuracy is', accuracy(y_test, y_pred))
print('Best model is', logreg_cv.best_params_)
cv_tabular(logreg_cv.cv_results_)

Fitting 2 folds for each of 36 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   39.2s
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 12.9min
[Parallel(

My best accuracy is 0.89232
Best model is {'clf__penalty': 'l2', 'vect__binary': False, 'vect__max_features': None, 'vect__ngram_range': (1, 1)}


,clf__penalty,vect__binary,vect__max_features,vect__ngram_range,mean_test_score,std_test_score,mean_fit_time
20,l1,True,NaN,"(3, 3)",0.774080,0.001363,28.247177
29,l1,False,NaN,"(3, 3)",0.774827,0.001411,28.160753
32,l1,False,50000.0,"(3, 3)",0.775520,0.001523,18.211167
35,l1,False,100000.0,"(3, 3)",0.775733,0.002269,18.462948
26,l1,True,100000.0,"(3, 3)",0.775787,0.000723,18.281320
23,l1,True,50000.0,"(3, 3)",0.776587,0.001523,18.369485
5,l2,True,50000.0,"(3, 3)",0.819467,0.001353,18.052719
14,l2,False,50000.0,"(3, 3)",0.820107,0.003166,18.169940
8,l2,True,100000.0,"(3, 3)",0.826133,0.002792,18.140384
17,l2,False,100000.0,"(3, 3)",0.827040,0.002952,18.426915


In [0]:
logreg_result = cv_tabular(logreg_cv.cv_results_)
csv_exp(logreg_result, 'Logreg')

## Decision Tree Estimator

In [0]:
dec_tree = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('norm', Normalizer()),
    ('clf-tree', tree.DecisionTreeClassifier())
])

In [0]:
params_dt = {"vect__ngram_range": [(1,1),(2,2)],
             "vect__max_features":[None, 10000, 50000],
             "clf-tree__max_depth": [None, 500, 5000]}
dec_tree_cv = GridSearchCV(dec_tree, param_grid = params_dt, cv=2, verbose = 10, n_jobs = -1)
dec_tree_cv.fit(X_train, y_train)
y_pred = dec_tree_cv.predict(X_test)
print('My accuracy is', accuracy(y_test, y_pred))
print('Best model is', dec_tree_cv.best_params_)
dt_result = cv_tabular(dec_tree_cv.cv_results_)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.2min
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 12.7min finished


My accuracy is 0.69552
Best model is {'clf-tree__max_depth': None, 'vect__max_features': 10000, 'vect__ngram_range': (1, 1)}


In [0]:
csv_exp(dt_result, 'dectree')
cv_tabular(dec_tree_cv.cv_results_)

,clf-tree__max_depth,vect__max_features,vect__ngram_range,mean_test_score,std_test_score,mean_fit_time
1,NaN,NaN,"(2, 2)",0.642187,0.004252,91.414668
13,5000.0,NaN,"(2, 2)",0.647573,0.003878,92.497577
7,500.0,NaN,"(2, 2)",0.648533,0.004197,91.646705
9,500.0,10000.0,"(2, 2)",0.652533,0.004677,22.837389
15,5000.0,10000.0,"(2, 2)",0.654293,0.006437,22.872181
3,NaN,10000.0,"(2, 2)",0.654453,0.002330,22.857502
17,5000.0,50000.0,"(2, 2)",0.656107,0.005263,34.245579
11,500.0,50000.0,"(2, 2)",0.656853,0.005583,33.407316
5,NaN,50000.0,"(2, 2)",0.657120,0.008623,33.167261
16,5000.0,50000.0,"(1, 1)",0.683040,0.000834,18.165090


## SVM Estimator

In [0]:
lsvm = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('norm', Normalizer()),
    ('clf', LinearSVC(random_state = 0, tol = 1e-7)),
])



params_svm = {"vect__ngram_range": [(1,1), (1,2),(2,2),(3,3)],
              "vect__binary":[True, False],
              "vect__max_features":[None, 50000, 100000],
              "clf__penalty": ["l2"]}
lsvm_cv = GridSearchCV(lsvm, param_grid = params_svm, cv=2, verbose = 10, n_jobs = -1)
lsvm_cv.fit(X_train, y_train)
y_pred = lsvm_cv.predict(X_test)
print('My best accuracy is', accuracy(y_test, y_pred))
print('Best model is', lsvm_cv.best_params_)
cv_tabular(lsvm_cv.cv_results_)

In [0]:
lsvm_result = cv_tabular(lsvm_cv.cv_results_)
csv_exp(lsvm_result, 'SVM')
lsvm_cv.cv_results_

## Naive Bayes

In [0]:
class NaiveBayes:
  def __init__(self, X_train, y_train):
    self.X_train = X_train
    self.y_train = y_train
    self.count_vect = None
    self.X_train_counts = None
    
    #prior
    self.pos_count = -1
    self.neg_count = -1
    self.doc_count = -1
    self.prior_log_pos = -1
    self.prior_log_neg = -1
    
    #In how many documents among all Pos corpus/ Neg corpus that we see a word
    self.occurrence_count_pos = None
    self.occurrence_count_neg = None
    
    #Log likelihood for each text feature
    self.feature_log_likelihood_matrix_pos = None
    self.feature_log_likelihood_matrix_neg = None
    self.feature_log_likelihood_matrix_not_occurred_pos = None
    self.feature_log_likelihood_matrix_not_occurred_neg = None
    
    #The sum of P(Xi=0|y=1), The sum of P(Xi=0|y=0)
    self.sum_feature_log_likelihood_matrix_not_occurred_pos = 0
    self.sum_feature_log_likelihood_matrix_not_occurred_neg = 0
  
  '''Get bag of words in order to calculate binary occurrence'''
  def get_BOW(self, X_train):
    self.count_vect = CountVectorizer(binary=False).fit(X_train)
    X_train_counts = self.count_vect.transform(X_train)
    return X_train_counts
  
  #log(theta)
  def log_likelihood_features(self, occurrence_count,y_count, doc_count):
    vfunc = np.vectorize(lambda x: math.log(float(x+1)/(y_count+2)))
    occurrence_count = vfunc(occurrence_count)
    return occurrence_count
  
  #log(1-theta)
  def log_likelihood_features_not_occurred(self, occurrence_count,y_count, doc_count):
    vfunc = np.vectorize(lambda x: math.log(1-(float(x+1)/(y_count+2))))
    occurrence_count = vfunc(occurrence_count)
    return occurrence_count
  
  def X_train_counts_filter_row(self,y_train_filtered, index_dict, X_train_counts):
    return X_train_counts[[index_dict[document_id] for document_id in y_train_filtered.index.values.tolist()]]
    
  def get_occurrence_count(self):
    index_dict = dict(zip(self.y_train.index.values.tolist(), range(self.X_train_counts.shape[0])))
    
    X_train_counts_pos = self.X_train_counts_filter_row(
        self.y_train.loc[y_train == 1],
        index_dict,
        self.X_train_counts)

    X_train_counts_neg = self.X_train_counts_filter_row(
        self.y_train.loc[y_train == 0],
        index_dict,
        self.X_train_counts)
    #occurrence_count: the counts of all non-zero entry of each column in the sparse matrix
    self.occurrence_count_pos = (X_train_counts_pos != 0).sum(0)
    self.occurrence_count_neg = (X_train_counts_neg != 0).sum(0)

  def train(self):
    self.pos_count =  len(y_train[y_train==1])
    self.neg_count = len(y_train[y_train==0])
    self.doc_count = self.pos_count + self.neg_count
    
    self.X_train_counts = self.get_BOW(self.X_train)

    self.prior_log_pos = math.log(self.pos_count/self.doc_count)
    self.prior_log_neg = math.log(self.neg_count/self.doc_count)

    self.get_occurrence_count()
    self.feature_log_likelihood_matrix_pos = self.log_likelihood_features(self.occurrence_count_pos,self.pos_count,self.doc_count)
    self.feature_log_likelihood_matrix_neg = self.log_likelihood_features(self.occurrence_count_neg,self.neg_count,self.doc_count)

    self.feature_log_likelihood_matrix_not_occurred_pos = self.log_likelihood_features_not_occurred(self.occurrence_count_pos,self.pos_count,self.doc_count)
    self.feature_log_likelihood_matrix_not_occurred_neg = self.log_likelihood_features_not_occurred(self.occurrence_count_neg,self.neg_count,self.doc_count)

    self.sum_feature_log_likelihood_matrix_not_occurred_pos = self.feature_log_likelihood_matrix_not_occurred_pos.sum()
    self.sum_feature_log_likelihood_matrix_not_occurred_neg = self.feature_log_likelihood_matrix_not_occurred_neg.sum()

  def predict(self,X_test):
    y_pred = []
    X_test_counts = self.count_vect.transform(X_test)
    nrows, ncols = X_test_counts.shape
    process = 0
    for row in X_test_counts:
      likelihood_pos_occur = 0
      likelihood_neg_occur = 0
      likelihood_pos_not_occur = self.sum_feature_log_likelihood_matrix_not_occurred_pos
      likelihood_neg_not_occur = self.sum_feature_log_likelihood_matrix_not_occurred_neg

      for ele in row:
        for word_index in ele.indices:
          likelihood_pos_occur += self.feature_log_likelihood_matrix_pos.item(word_index)
          likelihood_neg_occur += self.feature_log_likelihood_matrix_neg.item(word_index)
          likelihood_pos_not_occur -= self.feature_log_likelihood_matrix_not_occurred_pos.item(word_index)
          likelihood_neg_not_occur -= self.feature_log_likelihood_matrix_not_occurred_neg.item(word_index)

      if((likelihood_pos_occur+likelihood_pos_not_occur)
         >(likelihood_neg_occur+likelihood_neg_not_occur)): y_pred.append(1)
      else: y_pred.append(0)
      process += 1
      print('\r %d / %d '%(process,nrows), end='')
    return y_pred
           
    
nb_estimator = NaiveBayes(X_train,y_train)
nb_estimator.train()
y_pred = nb_estimator.predict(X_test)
print("accuracy on validation set for naive bayes is ", accuracy(y_test, y_pred))
  

 6250 / 6250 accuracy on validation set for naive bayes is  0.85328


# Output result to csv

In [0]:

def csv_make(model):
  result = df.loc[df['sentiment']>2]
  pred = model.predict(result['text'])
  
  a = {'Id':result['sentiment'], 'Category':pred}
  output = pd.DataFrame(data = a)
  output = output[['Id', 'Category']]
  output['Id'] = output['Id'].map(lambda x:x-100000)
  output = output.sort_values(by=['Id'])
  csv = output.to_csv(index=False)
  f = open('prediction.csv','w')
  f.write(csv)
  f.close()
  return output


csv_make(lsvm_cv)

,Id,Category
6965,0,0
12063,1,0
17179,2,0
19584,3,0
21023,4,1
7394,5,0
23997,6,0
824,7,0
19247,8,0
10654,9,0
